# generator

### Generators and yield
if a function uses the yield keyword, it defines an object known as generator whose primary use is to produce values for use in iteration. 

In [6]:
def countdown(n):
    print('Counting down from', n)
    while n>0:
        yield n
        n-=1
for x in countdown(10):
    print(x)
a=sum(countdown(10))
print(a)

Counting down from 10
10
9
8
7
6
5
4
3
2
1
Counting down from 10
55


In [3]:
c=countdown(10)
print(c)

<generator object countdown at 0x7abb74c572a0>


In [4]:
print(next(c))
print(next(c))
print(next(c))


Counting down from 10
10
9
8


next() statement makes the generator execute statements until it reaches yield statement, which returns a result and point of execution of fucntion is suspended until next() is invoked again. While it's suspended, the function retains all of its local vairables and execution environment. When resumed, execution continues with the statement follwoing the yield. 

a generator function produces items until it returns--- by reaching the end of the function or by using a return statement. this raises a StopIteration exception that terminates a for loop. If a generator function returns a non-None value, it is attached to the StopIteration exception. 

In [8]:
def func():
    yield 37
    return 41
f=func()
print(f)
try:
    print(next(f))
    print(next(f))
except StopIteration as e:
    value=e.value
    print(value)

<generator object func at 0x7abb7476c3b0>
37
41


If a generator is only partially consumed, the genrator function should perform some kind of cleanup action, make sure you sue ty-finally or a context manager. 

In [10]:
def countdown(n):
    print('counting down from', n)
    try: 
        while n>0:
            yield n
            n-=1
    finally:
        print('Only made it to', n)
for  n in countdown(10):
    if n==5:
        break
    print(n)

counting down from 10
10
9
8
7
6
Only made it to 5


### Restartable generators
If you want an object that allows repeated iteration, define it as a class and make the \_\_iter\_\_() method a generator.

In [11]:
c=countdown(3)
for n in c:
    print(n)
for n in c:
    print(n)

counting down from 3
3
2
1
Only made it to 0


In [12]:
class countdown:
    def __init__(self, start):
        self.start=start
    def __iter__(self):
        n=self.start
        while n>0:
            yield n
            n-=1
c=countdown(3)
for n in c:
    print(n)
for n in c:
    print(n)

3
2
1
3
2
1


### Generator Delegation
Generator has to be driven by using a for loop or explicit next() calls. Simply calling it is not enough, to address this, the yield from statement can be used. 

In [14]:
def countup(stop):
    n=1
    while n<=stop:
        yield n
        n+=1
def countdown(start):
    n=start
    while n>0:
        yield n
        n-=1
def up_and_down(n):
    yield from countup(n)
    yield from countdown(n)

for x in up_and_down(4):
    print(x, end=' ')

1 2 3 4 4 3 2 1 

In [15]:
def flatten(items):
    for i in items:
        if isinstance(i, list):
            yield from flatten(i)
        else:
            yield i
a=[1,2, [3,4], 6, [7,[8,9]]]
for x in flatten(a):
    print(x, end=' ')

1 2 3 4 6 7 8 9 

To bypass python's recursion limit and make it work with deeply nested structure, the iteration can be driven in a different way using a stack

In [ ]:
def flatten(items):
    stack=[iter(items)]
    while stack:
        try: 
            item=next(stack[-1])
            if isinstance(item, list):
                stack.append(iter(item))
            else:
                yield item
        except StopIteration:
            stack.pop()

### enchanced generator and yield expression
Yield statement can also be used as an expression that appears on the right hand side of an assignment operator such that instead of producing values, it executes in response to values sent to it. Python requires that the first send() call must send None, because the generator hasn’t paused yet.

In [21]:
def receiver():
    print("ready to receive")
    while True:
        n=yield
        print('got', n)
r=receiver()
r.send(None)
r.send("hello")
r.close()

# r.send(1)

ready to receive
got hello


Exceptions can be raised inside a generator using the throw(ty[,val[,tb]]) method with ty is exception type, val is exception argument or tuple of arguemtns and tb is an optional traceback. 


In [22]:
r=receiver()
r.throw(RuntimeError, "Dead")

/tmp/ipykernel_15424/3426940217.py:2: DeprecationWarning: the (type, exc, tb) signature of throw() is deprecated, use the single-arg signature instead.
  r.throw(RuntimeError, "Dead")


RuntimeError: Dead

### Applications of Enchanced Generator


In [29]:
def line_receiver():
    data=bytearray()
    line=None
    linecount=0
    while True:
        part=yield line
        linecount +=part.count(b'\n')
        data.extend(part)
        if linecount>0:
            index=data.index(b'\n')
            line=bytes(data[:index+1])
            data=data[index+1:]
            linecount-=1
        else:
            line=None
r = line_receiver()
print(r.send(None))             #

print(r.send(b'Hello'))        

print(r.send(b'world\nit'))   

print(r.send(b'WORKS\n')) 

None
None
b'Helloworld\n'
b'itWORKS\n'
